# Дополнительные установки 

In [ ]:
# Работа с табличными данными
!pip install pandas

In [ ]:
# Обучение моделей, кодирование категориальных признаков
!pip install scikit-learn

In [ ]:
# Распараллеливание операций с большими данными
!pip install dask

In [ ]:
# Обучение модели Word2Vec для эмбеддингов
!pip install gensim

In [ ]:
# Загрузка датасета
!pip install kagglehub 

In [ ]:
# Создание и обучение нейронных сетей, работа с тензорами
!pip install torch

In [ ]:
# Добавление прогресс-бара в цикл
!pip install tqdm

In [ ]:
# Оптимизация гиперпараметров модели 
!pip install optuna

# Предобработка данных

In [ ]:
import pandas as pd
import numpy as np
import kagglehub
from datetime import datetime, timedelta

In [ ]:
# Загрузка данных
path = kagglehub.dataset_download("retailrocket/ecommerce-dataset")
events = pd.read_csv(path+'/events.csv')
item_properties_p1 = pd.read_csv(path+'/item_properties_part1.csv')
item_properties_p2 = pd.read_csv(path+'/item_properties_part2.csv')
item_properties = pd.concat([item_properties_p1, item_properties_p2])

In [ ]:
print(events.head(5))
print(item_properties.head(5))

In [ ]:
events.dtypes

In [ ]:
item_properties.dtypes

In [ ]:
# Очистка данных (дубли/null)
events = events.drop_duplicates()
item_properties = item_properties.drop_duplicates()
events = events.dropna(subset = ['event'])
events = events.dropna(subset = ['itemid'])
events = events.dropna(subset = ['visitorid'])
events = events.dropna(subset = ['timestamp'])
item_properties = item_properties.dropna(subset = ['itemid'])
item_properties = item_properties.dropna(subset = ['timestamp'])

# Приведение timestamp к единому типу данных для merge
events['timestamp'] = events['timestamp'].astype('int64')
item_properties['timestamp'] = item_properties['timestamp'].astype('int64')
events = events.sort_values(['timestamp']).reset_index(drop=True)
item_properties = item_properties.sort_values(['timestamp']).reset_index(drop=True)

In [ ]:
# Соединение events и item_properties
merged_data = pd.merge_asof(
    events,
    item_properties,
    on='timestamp',
    by='itemid',
    direction='backward'
)
# Проверка что все правильно соединилось без дублей
print("Было:", len(events), ",стало:", len(merged_data))

In [ ]:
# Создание сессий (действия пользователя в течение 30 минут)
merged_data = merged_data.drop(['transactionid'], axis = 1)
merged_data['property'] = merged_data['property'].fillna(0)
merged_data['value'] = merged_data['value'].fillna(0)
merged_data['timestamp'] = pd.to_datetime(merged_data['timestamp'], unit='ms')
merged_data = merged_data.sort_values(by=['visitorid', 'timestamp'])

merged_data['session_id'] = (
    (merged_data['timestamp'].diff() >= pd.Timedelta(minutes=30)) | 
    (merged_data['visitorid'] != merged_data['visitorid'].shift())
).cumsum()

print(merged_data.head(5))
print("Уникальных сессий:", merged_data['session_id'].nunique())
print("Уникальных пользователей:", merged_data['visitorid'].nunique())

In [ ]:
from gensim.models import Word2Vec
from sklearn.preprocessing import LabelEncoder

In [ ]:
# Создание эмбеддингов property/value
merged_data['property_value'] = merged_data['property'].astype(str) + ':' + merged_data['value'].astype(str)
property_value_texts = merged_data.groupby('itemid')['property_value'].apply(lambda x: ' '.join(x)).values
property_value_tokens = [text.split() for text in property_value_texts]
property_value_model = Word2Vec(sentences=property_value_tokens, vector_size=16, window=5, min_count=1, sg=1, epochs=10)
property_value_embeddings = {word: property_value_model.wv[word] for word in property_value_model.wv.index_to_key}

def get_item_embedding(item_properties):
    embeddings = [property_value_embeddings[prop] for prop in item_properties if prop in property_value_embeddings]
    if embeddings:
        return np.mean(embeddings, axis=0)
    else:
        return np.zeros(32)

item_embeddings = {itemid: get_item_embedding(text.split()) for itemid, text in zip(merged_data['itemid'].unique(), property_value_texts)}
item_embeddings_df = pd.DataFrame.from_dict(item_embeddings, orient='index')
item_embeddings_df.reset_index(inplace=True)
item_embeddings_df.rename(columns={'index': 'itemid'}, inplace=True)
embedding_columns = list(item_embeddings_df.columns)

# Присоединение эмбеддингов к основной таблице
merged_data = merged_data.merge(item_embeddings_df, on='itemid', how='left')

# Создание эмбеддингов event
event_encoder = LabelEncoder()
merged_data['event_encoded'] = event_encoder.fit_transform(merged_data['event'])
print(merged_data.head(5))

In [ ]:
import dask.dataframe as dd
import os
from tqdm import tqdm
import dask

In [ ]:
dask_data = dd.from_pandas(merged_data, npartitions=100)
# Папка для .npz файлов
os.makedirs("files_npz", exist_ok=True)

def create_sequence_for_session(session_df):
    """
    Функция для создания последовательностей признаков и меток для сессии.
    Принимает DataFrame с данными одной сессии и возвращает X (последовательности признаков)
    и y (метки событий).

    Параметры:
    session_df (pd.DataFrame): DataFrame, содержащий данные для одной сессии.

    Возвращает:
    tuple или None: Возвращает кортеж (X, y) с последовательностями признаков и метками,
    если последовательности были успешно созданы, иначе возвращает None.
    """
    session_df = session_df.sort_values('timestamp')
    features = session_df.iloc[:, 10:].values 
    events = session_df['event_encoded'].values.reshape(-1, 1)
    X, y = [], []
    seq_length = 10
    for i in range(len(features) - seq_length):
        X.append(features[i:i + seq_length])
        y.append(events[i + seq_length])      
    if len(X) > 0:
        return np.array(X), np.array(y)
    else:
        return None

df = dask_data.compute()
groups = list(df.groupby(['visitorid', 'session_id']))

# Цикл по всем группам 'visitorid' + 'session_id' и сохранение данных в .npz файлы
for (visitor_id, session_id), group in tqdm(groups, desc="Saving .npz files"):
    if len(group) >= 10:
        result = create_sequence_for_session(group)
        if result is not None:
            X, y = result
            filename = f"session_{visitor_id}_{session_id}.npz"
            filepath = os.path.join("files_npz", filename)
            np.savez_compressed(filepath, X=X, y=y)

In [ ]:
# Проверка что dask-файлы создались
directory = os.path.join(os.getcwd(), "files_npz")
for root, dirs, files in os.walk(directory):
    for file in files:
        if file.endswith(".npz"):
            print("Найден файл:", os.path.join(root, file))
            break

In [ ]:
# Данные из dask-файлов -> признаки и метки для обучения
def load_data(directory):
    X, y = [], []
    for file in os.listdir(directory):
        if file.endswith(".npz"):
            data = np.load(os.path.join(directory, file))
            X.append(data['X'])
            y.append(data['y'])
    return np.concatenate(X, axis=0), np.concatenate(y, axis=0)

directory = os.path.join(os.getcwd(), "files_npz")
X, y = load_data(directory)
                               
# Проверка на корректный тип данных и размерности
print(f"Тип X: {type(X)}")
print(f"Тип y: {type(y)}")
print(f"Размер X: {X.shape}")
print(f"Размер y: {y.shape}")

In [ ]:
# Приведение последовательностей к одинаковой длине
seq_length = 10

def custom_pad_sequences(X, seq_length):
    padded_sequences = []
    
    for seq in X:
        if len(seq) < seq_length:
            padding = np.zeros((seq_length - len(seq), seq.shape[1]))
            padded_seq = np.vstack([seq, padding])
        else:
            padded_seq = seq[:seq_length]
        padded_sequences.append(padded_seq)
    
    return np.array(padded_sequences)

X_padded = custom_pad_sequences(X, seq_length)
print(f"X_padded.shape: {X_padded.shape}")

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X_padded,
                y,stratify=y, test_size=0.2, random_state=42)

print(f"Размер X_train: {X_train.shape}")
print(f"Размер X_test: {X_test.shape}")
print(f"Размер y_train: {y_train.shape}")
print(f"Размер y_test: {y_test.shape}")

# Модель LRCN (базовая)

In [ ]:
import torch
import time
from torch.utils.data import DataLoader, TensorDataset
import torch.nn as nn
from sklearn.metrics import precision_score, recall_score, f1_score, confusion_matrix

In [ ]:
# Преобразование данных в тензоры для использования в модели
X_train_tensor = torch.tensor(X_train, dtype=torch.float32)
X_test_tensor = torch.tensor(X_test, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train, dtype=torch.long).squeeze()
y_test_tensor = torch.tensor(y_test, dtype=torch.long).squeeze()
train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
test_dataset = TensorDataset(X_test_tensor, y_test_tensor)
train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False)

In [ ]:
"""
Класс модели
Наследуется от nn.Module (базовый класс для всех моделей в PyTorch)
"""
class LRCN(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, num_classes):
        super(LRCN, self).__init__()
        
        # CNN часть
        self.conv1 = nn.Conv1d(in_channels=input_size, out_channels=128, kernel_size=3, padding=1)
        self.conv2 = nn.Conv1d(in_channels=128, out_channels=256, kernel_size=3, padding=1)
        self.relu = nn.ReLU()
        self.pool = nn.MaxPool1d(kernel_size=2, stride=2)
        # LSTM часть
        self.lstm = nn.LSTM(input_size=256, hidden_size=hidden_size, num_layers=num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_size, num_classes)

    def forward(self, x):
        # (batch_size, seq_length, input_size) -> (batch_size, input_size, seq_length)
        x = x.permute(0, 2, 1)
        x = self.relu(self.conv1(x))
        x = self.pool(x)
        x = self.relu(self.conv2(x))
        x = self.pool(x) 
        x = x.permute(0, 2, 1)
        lstm_out, _ = self.lstm(x)
        lstm_out = lstm_out[:, -1, :]
        output = self.fc(lstm_out)
        
        return output

# Функция для обучения модели
def train_model(model, train_loader, criterion, optimizer, num_epochs=10):
    model.train()
    for epoch in range(num_epochs):
        total_loss = 0
        
        for X_batch, y_batch in train_loader:
            X_batch, y_batch = X_batch.to(device), y_batch.to(device)
            outputs = model(X_batch)
            loss = criterion(outputs, y_batch)
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            total_loss += loss.item()
        
        print(f"Эпоха [{epoch+1}/{num_epochs}], Потеря: {total_loss / len(train_loader):.4f}")

# Функция для оценки модели на тестовых данных
def evaluate_model(model, test_loader):
    model.eval()
    all_preds = []
    all_labels = []
    
    with torch.no_grad():
        for X_batch, y_batch in test_loader:
            X_batch, y_batch = X_batch.to(device), y_batch.to(device)
            outputs = model(X_batch)
            _, predicted = torch.max(outputs.data, 1)
            all_preds.extend(predicted.cpu().numpy())
            all_labels.extend(y_batch.cpu().numpy())
    
    accuracy = 100 * (sum(1 for p, l in zip(all_preds, all_labels) if p == l) / len(all_labels))
    precision = precision_score(all_labels, all_preds, average='weighted')
    recall = recall_score(all_labels, all_preds, average='weighted')
    f1 = f1_score(all_labels, all_preds, average='weighted')
    print(f"Accuracy: {accuracy:.2f}%")
    print(f"Precision: {precision:.4f}")
    print(f"Recall: {recall:.4f}")
    print(f"F1-Score: {f1:.4f}")

In [ ]:
# Объявление параметров модели
input_size = X_train.shape[2]
hidden_size = 128
num_layers = 2
num_classes = len(np.unique(y))
criterion = nn.CrossEntropyLoss()
# Объявление модели
lrcn_model = LRCN(input_size, hidden_size, num_layers, num_classes)
optimizer = torch.optim.Adam(lrcn_model.parameters(), lr=0.001)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
lrcn_model.to(device)
# Обучение и оценка модели
start_time = time.time()
train_model(lrcn_model, train_loader, criterion, optimizer, num_epochs=10)
evaluate_model(lrcn_model, test_loader)
end_time = time.time()
print(f"Время работы: {end_time - start_time:.2f} секунд")

# Улучшенная модель

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import DataLoader, TensorDataset, random_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.model_selection import train_test_split
import optuna
import time

In [ ]:
# Преобразование данных в тензоры
X_train_tensor = torch.tensor(X_train, dtype=torch.float32)
X_test_tensor = torch.tensor(X_test, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train, dtype=torch.long).squeeze()
y_test_tensor = torch.tensor(y_test, dtype=torch.long).squeeze()
train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
test_dataset = TensorDataset(X_test_tensor, y_test_tensor)

# Деление train_dataset на train и val
val_size = int(0.2 * len(train_dataset))
train_size = len(train_dataset) - val_size
train_dataset, val_dataset = random_split(train_dataset, [train_size, val_size])
train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=64, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False)

In [ ]:
"""
Класс модели
Наследуется от nn.Module (базовый класс для всех моделей в PyTorch)
"""
class ImprovedLRCN(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, num_classes, dropout=0.3):
        super(ImprovedLRCN, self).__init__()

        # CNN часть
        self.conv1 = nn.Conv1d(in_channels=input_size, out_channels=128, kernel_size=3, padding=1)
        self.conv2 = nn.Conv1d(in_channels=128, out_channels=256, kernel_size=3, padding=1)
        self.norm1 = nn.LayerNorm(256)
        self.gelu = nn.GELU()
        self.dropout = nn.Dropout(dropout)
        # LSTM часть
        self.lstm = nn.LSTM(input_size=256, hidden_size=hidden_size, num_layers=num_layers,
                            batch_first=True, bidirectional=True)

        self.attention = nn.Linear(hidden_size * 2, 1)
        self.fc = nn.Linear(hidden_size * 2, num_classes)

    def forward(self, x):
        x = x.permute(0, 2, 1)
        x = self.gelu(self.conv1(x))
        x = self.gelu(self.conv2(x))
        x = x.permute(0, 2, 1)
        x = self.norm1(x)
        x = self.dropout(x)
        lstm_out, _ = self.lstm(x)
        attention_weights = torch.softmax(self.attention(lstm_out), dim=1)
        context_vector = torch.sum(attention_weights * lstm_out, dim=1)
        output = self.fc(context_vector)
        return output

# Функция для обучения модели
def train_model(model, train_loader, criterion, optimizer, scheduler, num_epochs=10):
    model.train()
    for epoch in range(num_epochs):
        total_loss = 0

        for X_batch, y_batch in train_loader:
            X_batch, y_batch = X_batch.to(device), y_batch.to(device)
            outputs = model(X_batch)
            loss = criterion(outputs, y_batch)
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            total_loss += loss.item()

        scheduler.step(total_loss / len(train_loader))
        print(f"Эпоха [{epoch + 1}/{num_epochs}], Потеря: {total_loss / len(train_loader):.4f}")

# Функция для оценки модели на тестовых данных
def evaluate_model(model, loader, criterion):
    model.eval()
    total_loss = 0
    all_preds, all_labels = [], []
    
    with torch.no_grad():
        for X_batch, y_batch in loader:
            X_batch, y_batch = X_batch.to(device), y_batch.to(device)
            outputs = model(X_batch)
            loss = criterion(outputs, y_batch)
            total_loss += loss.item()
            _, predicted = torch.max(outputs.data, 1)
            all_preds.extend(predicted.cpu().numpy())
            all_labels.extend(y_batch.cpu().numpy())

    avg_loss = total_loss / len(loader)
    accuracy = accuracy_score(all_labels, all_preds) * 100
    precision = precision_score(all_labels, all_preds, average='weighted')
    recall = recall_score(all_labels, all_preds, average='weighted')
    f1 = f1_score(all_labels, all_preds, average='weighted')

    return accuracy, precision, recall, f1

# Функция для подбора гиперпараметров -> увеличения эффективности
def objective(trial):
    hidden_size = trial.suggest_int('hidden_size', 64, 256)
    num_layers = trial.suggest_int('num_layers', 1, 3)
    dropout = trial.suggest_float('dropout', 0.1, 0.5)
    lr = trial.suggest_loguniform('lr', 1e-4, 1e-2)

    model = ImprovedLRCN(input_size=X_train.shape[2], hidden_size=hidden_size, num_layers=num_layers,
                         num_classes=len(np.unique(y)), dropout=dropout).to(device)

    criterion = nn.CrossEntropyLoss()
    optimizer = optim.AdamW(model.parameters(), lr=lr)
    scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, 'min', patience=2, factor=0.5)
    train_model(model, train_loader, criterion, optimizer, scheduler, num_epochs=5)
    _, _, _, f1 = evaluate_model(model, val_loader, criterion)

    trial.report(f1, 0)
    return f1

In [ ]:
"""
WARNING: работает очень долго (~1 час)
"""
# Подбор гиперпараметров 
study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=20)
best_params = study.best_params
print(f"Лучшие параметры: {best_params}")

In [ ]:
# Объявление параметров модели 
hidden_size = 125
num_layers = 3
dropout = 0.36597
lr = 0.0006
criterion = nn.CrossEntropyLoss()
# Объявление модели
improved_lrcn = ImprovedLRCN(input_size=X_train.shape[2], 
                    hidden_size=hidden_size, num_layers=num_layers,
                   num_classes=len(np.unique(y)), dropout=dropout)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
improved_lrcn.to(device)
optimizer = optim.AdamW(improved_lrcn.parameters(), lr=lr)
scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, 'min', patience=2, factor=0.5)
# Обучение и оценка модели
start_time = time.time()
train_model(improved_lrcn, train_loader, criterion, optimizer, 
            scheduler, num_epochs=10)
accuracy, precision, recall, f1 = evaluate_model(improved_lrcn, val_loader,
                                                 criterion)
end_time = time.time()
print(f"Accuracy: {accuracy:.2f}%")
print(f"Precision: {precision:.4f}")
print(f"Recall: {recall:.4f}")
print(f"F1-score: {f1:.4f}")
print(f"Время работы: {end_time - start_time:.2f} секунд")

# Сравнение с другими моделями

- Feedforward neural networks
- Recurrent neural networks with LSTM cells
- Transformer-based models

## FFNN 

In [ ]:
import torch
import time
import torch.nn as nn
import numpy as np
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

In [ ]:
"""
FFNN ожидает один фиксированный вектор признаков на каждый пример,
признаки с разными масштабами усложняют процесс обучения
Усреднение → превращаем последовательность в один вектор признаков
Стандартизация → делаем признаки "одинаковыми" для лучшего обучения
"""
X_train_ffnn = np.mean(X_train, axis=1)
X_test_ffnn = np.mean(X_test, axis=1)
scaler = StandardScaler()
X_train_ffnn = scaler.fit_transform(X_train_ffnn)
X_test_ffnn = scaler.transform(X_test_ffnn)

In [ ]:
# Преобразование данных в тензоры для использования в модели
X_train_tensor = torch.tensor(X_train_ffnn, dtype=torch.float32)
X_test_tensor = torch.tensor(X_test_ffnn, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train, dtype=torch.long).squeeze()
y_test_tensor = torch.tensor(y_test, dtype=torch.long).squeeze()
train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
test_dataset = TensorDataset(X_test_tensor, y_test_tensor)
train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False)

In [ ]:
"""
Класс модели
Наследуется от nn.Module (базовый класс для всех моделей в PyTorch)
"""
class FFNN(nn.Module):
    def __init__(self, input_size, hidden_size, num_classes):
        super(FFNN, self).__init__()
        self.model = nn.Sequential(
            nn.Flatten(),
            nn.Linear(input_size, hidden_size),
            nn.ReLU(),
            nn.Linear(hidden_size, hidden_size),
            nn.ReLU(),
            nn.Linear(hidden_size, num_classes)
        )

    def forward(self, x):
        return self.model(x)

# Функция для обучения модели
def train_model(model, train_loader, criterion, optimizer, num_epochs=10):
    train_losses = []

    for epoch in range(num_epochs):
        model.train()
        total_loss = 0

        for X_batch, y_batch in train_loader:
            X_batch, y_batch = X_batch.to(device), y_batch.to(device)
            optimizer.zero_grad()
            outputs = model(X_batch)
            loss = criterion(outputs, y_batch)
            loss.backward()
            optimizer.step()
            total_loss += loss.item()
        avg_loss = total_loss / len(train_loader)
        train_losses.append(avg_loss)
        print(f"Эпоха [{epoch+1}/{num_epochs}], Потеря: {avg_loss:.4f}")

# Функция для оценки модели на тестовых данных
def evaluate_model(model, test_loader):
    model.eval()
    all_preds = []
    all_labels = []

    with torch.no_grad():
        for X_batch, y_batch in test_loader:
            X_batch, y_batch = X_batch.to(device), y_batch.to(device)
            outputs = model(X_batch)
            _, predicted = torch.max(outputs, 1)
            all_preds.extend(predicted.cpu().numpy())
            all_labels.extend(y_batch.cpu().numpy())

    accuracy = accuracy_score(all_labels, all_preds)
    precision = precision_score(all_labels, all_preds, average='weighted')
    recall = recall_score(all_labels, all_preds, average='weighted')
    f1 = f1_score(all_labels, all_preds, average='weighted')

    print(f"Accuracy: {accuracy * 100:.2f}%")
    print(f"Precision: {precision:.4f}")
    print(f"Recall: {recall:.4f}")
    print(f"F1-Score: {f1:.4f}")

In [ ]:
# Объявление параметров модели
input_size = X_train_ffnn.shape[1] 
hidden_size = 128
num_classes = len(np.unique(y))
criterion = nn.CrossEntropyLoss()
# Объявление модели
ffnn_model = FFNN(input_size, hidden_size, num_classes)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
ffnn_model.to(device)
optimizer = optim.Adam(ffnn_model.parameters(), lr=0.001)
# Обучение и оценка модели
start_time = time.time()
train_model(ffnn_model, train_loader, criterion, optimizer, num_epochs=10)
evaluate_model(ffnn_model, test_loader)
end_time = time.time()
print(f"Время работы: {end_time - start_time:.2f} секунд")

## LSTM

In [ ]:
import torch
import time
import torch.nn as nn
import numpy as np
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

In [ ]:
# Преобразование данных в тензоры для использования в модели
X_train_tensor = torch.tensor(X_train, dtype=torch.float32)
X_test_tensor = torch.tensor(X_test, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train, dtype=torch.long).squeeze()
y_test_tensor = torch.tensor(y_test, dtype=torch.long).squeeze()
train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
test_dataset = TensorDataset(X_test_tensor, y_test_tensor)
train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False)

In [ ]:
"""
Класс модели
Наследуется от nn.Module (базовый класс для всех моделей в PyTorch)
"""
class LSTM(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, num_classes):
        super(LSTM, self).__init__()
        self.lstm = nn.LSTM(input_size=input_size, hidden_size=hidden_size, num_layers=num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_size, num_classes)

    def forward(self, x):
        lstm_out, _ = self.lstm(x)
        lstm_out = lstm_out[:, -1, :]  # Используем последний выход LSTM
        out = self.fc(lstm_out)
        return out
    
# Функция для обучения модели    
def train_model(model, train_loader, criterion, optimizer, num_epochs=10):
    model.train()
    for epoch in range(num_epochs):
        total_loss = 0

        for X_batch, y_batch in train_loader:
            X_batch, y_batch = X_batch.to(device), y_batch.to(device)
            outputs = model(X_batch)
            loss = criterion(outputs, y_batch)
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            total_loss += loss.item()

        print(f"Эпоха [{epoch + 1}/{num_epochs}], Потеря: {total_loss / len(train_loader):.4f}")

# Функция для оценки модели на тестовых данных
def evaluate_model(model, test_loader):
    model.eval()
    all_preds = []
    all_labels = []

    with torch.no_grad():
        for X_batch, y_batch in test_loader:
            X_batch, y_batch = X_batch.to(device), y_batch.to(device)
            outputs = model(X_batch)
            _, predicted = torch.max(outputs, 1)
            all_preds.extend(predicted.cpu().numpy())
            all_labels.extend(y_batch.cpu().numpy())

    accuracy = accuracy_score(all_labels, all_preds)
    precision = precision_score(all_labels, all_preds, average='weighted')
    recall = recall_score(all_labels, all_preds, average='weighted')
    f1 = f1_score(all_labels, all_preds, average='weighted')

    print(f"Accuracy: {accuracy * 100:.2f}%")
    print(f"Precision: {precision:.4f}")
    print(f"Recall: {recall:.4f}")
    print(f"F1-Score: {f1:.4f}")


In [ ]:
# Объявление параметров модели
input_size = X_train.shape[2]
hidden_size = 128
num_layers = 2
num_classes = len(np.unique(y))
criterion = nn.CrossEntropyLoss()
# Объявление модели
lstm_model = LSTM(input_size, hidden_size, num_layers, num_classes)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
lstm_model.to(device)
optimizer = torch.optim.Adam(lstm_model.parameters(), lr=0.001)
# Обучение и оценка модели
start_time = time.time()
train_model(lstm_model, train_loader, criterion, optimizer, num_epochs=10)
evaluate_model(lstm_model, test_loader)  
end_time = time.time()
print(f"Время работы: {end_time - start_time:.2f} секунд")

# Transformer

In [ ]:
import torch
import torch.nn as nn
from torch.utils.data import DataLoader, TensorDataset
import numpy as np
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
import time

In [ ]:
# Преобразование данных в тензоры для использования в модели
X_train_tensor = torch.tensor(X_train, dtype=torch.float32)
X_test_tensor = torch.tensor(X_test, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train, dtype=torch.long).squeeze()
y_test_tensor = torch.tensor(y_test, dtype=torch.long).squeeze()
train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
test_dataset = TensorDataset(X_test_tensor, y_test_tensor)
train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False)

In [ ]:
"""
Класс модели
Наследуется от nn.Module (базовый класс для всех моделей в PyTorch)
"""
class TransformerClassifier(nn.Module):
    def __init__(self, feature_dim, d_model, nhead, num_layers, num_classes, dim_feedforward=128, dropout=0.1):
        super(TransformerClassifier, self).__init__()
        self.embedding = nn.Linear(feature_dim, d_model)
        encoder_layer = nn.TransformerEncoderLayer(d_model=d_model, nhead=nhead, dim_feedforward=dim_feedforward, dropout=dropout, batch_first=True)
        self.transformer_encoder = nn.TransformerEncoder(encoder_layer, num_layers=num_layers)
        self.classifier = nn.Linear(d_model, num_classes)

    def forward(self, x):
        x = self.embedding(x)  # [batch_size, seq_len, d_model]
        x = self.transformer_encoder(x)  # [batch_size, seq_len, d_model]
        x = x.mean(dim=1)  # Average pooling over time dimension
        out = self.classifier(x)  # [batch_size, num_classes]
        return out

# Функция для обучения модели
def train_model(model, train_loader, criterion, optimizer, num_epochs=10):
    model.train()
    for epoch in range(num_epochs):
        total_loss = 0
        for X_batch, y_batch in train_loader:
            X_batch, y_batch = X_batch.to(device), y_batch.to(device)
            outputs = model(X_batch)
            loss = criterion(outputs, y_batch)
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            total_loss += loss.item()
        print(f"Эпоха [{epoch + 1}/{num_epochs}], Потеря: {total_loss / len(train_loader):.4f}")

# Функция для оценки модели на тестовых данных
def evaluate_model(model, test_loader):
    model.eval()
    all_preds = []
    all_labels = []
    with torch.no_grad():
        for X_batch, y_batch in test_loader:
            X_batch, y_batch = X_batch.to(device), y_batch.to(device)
            outputs = model(X_batch)
            _, predicted = torch.max(outputs, 1)
            all_preds.extend(predicted.cpu().numpy())
            all_labels.extend(y_batch.cpu().numpy())
    accuracy = accuracy_score(all_labels, all_preds)
    precision = precision_score(all_labels, all_preds, average='weighted')
    recall = recall_score(all_labels, all_preds, average='weighted')
    f1 = f1_score(all_labels, all_preds, average='weighted')
    print(f"Accuracy: {accuracy * 100:.2f}%")
    print(f"Precision: {precision:.4f}")
    print(f"Recall: {recall:.4f}")
    print(f"F1-Score: {f1:.4f}")

In [ ]:
# Объявление параметров модели
sequence_length = X_train.shape[1] 
feature_dim = X_train.shape[2]      
num_classes = len(np.unique(y_train_tensor.numpy()))
d_model = 64 
nhead = 4
num_layers = 2
dim_feedforward = 128
dropout = 0.1
criterion = nn.CrossEntropyLoss()
# Объявление модели
transformer_model = TransformerClassifier(feature_dim=feature_dim,
    d_model=d_model, nhead=nhead, num_layers=num_layers,
    num_classes=num_classes, dim_feedforward=dim_feedforward,
    dropout=dropout
)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
transformer_model.to(device)
optimizer = torch.optim.Adam(transformer_model.parameters(), lr=0.001)
# Обучение и оценка модели
start_time = time.time()
train_model(transformer_model, train_loader, criterion, optimizer, num_epochs=10)
evaluate_model(transformer_model, test_loader)
end_time = time.time()
print(f"Время работы: {end_time - start_time:.2f} секунд")